In [1]:
%matplotlib inline

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

from simple_movement import DataGen, SimpleMovement

In [19]:
class Encoder(nn.Module):
    
    def __init__(self, num_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channel, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv4 = nn.Conv2d(8, 8, 2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(200, 128)
        self.fc2 = nn.Linear(128, 3)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 5*5*8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
def get_model():
    
    net = Encoder(num_channel=1)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    return net, optimizer

In [26]:
def to_tensor(x):
    return torch.from_numpy(x).float()

def get_data(height, width, num_channel, path, size, bs=10):
    
    dgen = DataGen(height, width, num_channel)
    x, y = dgen.get_feature_target_pairs(path, size)
    print('Data loaded...\nx:{}\ty:{}\n'.format(x.shape, y.shape))
    
    x = x / 255.
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=331)
    x_train, x_test, y_train, y_test = map(to_tensor, (x_train, x_test, y_train, y_test))
    
    train_ds = TensorDataset(x_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

    test_ds = TensorDataset(x_test, y_test)
    test_dl = DataLoader(test_ds, batch_size=bs)
    
    return train_dl, test_dl

In [27]:
def fit(net, optimizer, train_dl, test_dl, epochs=10):
    
    loss_function = nn.MSELoss()
    for epoch in range(epochs):
        net.train()
        for x, y in train_dl:
            pred = net(x)
            loss = loss_function(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        net.eval()
        with torch.no_grad():
            test_loss = sum(loss_function(net(x), y) for x, y in test_dl)
        print('epoch:{}, test_loss:{}'.format(epoch+1, test_loss/len(test_dl)))

In [28]:
net, optimizer = get_model()
train_dl, test_dl = get_data(100, 100, 1, './data/data_simple_movement_1/', 8)
fit(net, optimizer, train_dl, test_dl)

Data loaded...
x:(96, 1, 100, 100)	y:(96, 3)

epoch:0, test_loss:6845.8974609375
epoch:1, test_loss:6724.69921875
epoch:2, test_loss:6149.25830078125
epoch:3, test_loss:4067.844482421875
epoch:4, test_loss:2814.79736328125
epoch:5, test_loss:2366.677734375
epoch:6, test_loss:2219.5302734375
epoch:7, test_loss:2488.86767578125
epoch:8, test_loss:2264.97021484375
epoch:9, test_loss:2391.91162109375
